In [ ]:
import pandas as pd             
import numpy as np 
import matplotlib.pyplot as plt   
%matplotlib inline

# Data Preprocessing

First we will do some data preprocessing to make the dataset reading for training. Indeed by reading through the example notebook and by further analysis. Some inconsitencies will be problematic durint the training of the random forest.

In [ ]:
# Load dataset
df = pd.read_csv('CrowdstormingDataJuly1st.csv')
df['rater1'] = df['rater1'].astype('category')
df['rater2'] = df['rater2'].astype('category')

df.head(3)

The first step is to remove all the rows that from which we can not infere the answer to the question asked. We need to find the skin color of the soccer player based on the other feature at our disposal. We will thus first take interest in the column '_rater1_' and '_rater2_'. By looking at the dataset we can directly see that some rows have no "rating" and won't be able to help use during the training: we don't have the output labels for the classifier.

We only have rating for ~85% of the dataset and each sample is unique.

In [ ]:
# Good news we always have either the two rating or none
sum(~(df.rater1.isnull() == df.rater2.isnull()))

data = df[~df.rater1.isnull()].copy()
print('Total available', len(df))
print('Total with rating', len(data), "({}%)".format(round(len(data)*100/len(df),3)))
print('Number of sample with disagrement:', sum(~(data.rater1 == data.rater2)))

We can also look at own many "real" unique sample we have. Indeed a player is most certainly present multiple times. As we can see most of the players have several entries.

In [ ]:
print("Number of unique player", len(data.player.unique()))
data.player.value_counts().plot()

-----------